# Set up

Load relevant libraries

https://darwin.prod.linkedin.com/ui/notebook-detail/lab/?path=GTME%20scripts/GAI/Automation/Analysis/R%20analysis.ipynb

In [3]:
library(tidyverse) # dplyr, tidyr etc.
library(magrittr) # pipe function %>%
library(DT) # datatable (view data as table)
library(ggalt) # splines for curves
library(Hmisc)

Set colour scheme details and labels here then load data

In [4]:
li_orange = "#f18f01"
li_blue = "#006e90"
li_black = "#151a25"
li_grey = "#dddddd"
li_green = "green4"
li_red = "#D7161E"

cols <- c(li_black, li_blue, li_orange, li_grey)
cols2 <-  c(li_black, li_blue, li_orange, li_grey, li_red)
cs = colorRampPalette(cols)(50)

annotation <- data.frame(
   x = c(1.5,2.5,3.5,4.5),
   y = rep(87, 4),
   label = c(
     "Skill not\nDemonstrated",
     "Foundational",
     "Good",
     "Best in Class")
)


In [5]:
data <- read_csv("/tmp/rinput.csv",
                 col_types = cols(
                     callid = col_character(), 
                     calldate = col_date(format = "%Y-%m-%d"),
                     research_score = col_double(),
                     highimpactquestions_score = col_double(),
                     examinechallenges_score = col_double(),
                     presentimpactstory_score = col_double(),
                     sharekeyinsights_score = col_double(),
                     alignuniquedifferentiators_score = col_double(),
                     delivercompellingproposals_score = col_double(),
                     dealnegotiate_score = col_double(),
                     handleobjections_score = col_double(),
                     navigatedecisionmakingprocess_score = col_double(),
                     setpurposefulmilestones_score = col_double(),
                     establishalignmentaccountability_score = col_double()
                 ))

# Analysis

Number of distinct reps (hosts) and calls by region

In [60]:
Hmisc::describe(data)

In [61]:
data %>% 
  group_by(rep_region) %>% 
  summarise(
      reps = n_distinct(emailaddress),
      calls = n()
  ) %>% 
  rbind(
      c("Total", sum(.$reps), sum(.$calls))
  )

Best in class by subskill. Returns count and % of BIC skills as a fraction of total calls. Also shows % calls with at least 1 BIC skill.

In [62]:
data %>% 
    select(-c(crm_country, rep_country, rep_region, calldate, emailaddress)) %>% 
    mutate(
        BIC_deliver_the_business_case = case_when(
            (delivercompellingproposals_score >= 4 & handleobjections_score >= 4) |
            (delivercompellingproposals_score >= 4 & dealnegotiate_score >= 4) |
            (handleobjections_score >= 4 & dealnegotiate_score >= 4) ~ 1,
            TRUE ~ 0
        ),
        BIC_discovery = case_when(
            (examinechallenges_score >= 4 & highimpactquestions_score >= 4) |
            (examinechallenges_score >= 4 & research_score >= 4) |
            (highimpactquestions_score >= 4 & research_score >= 4) ~ 1,
            TRUE ~ 0
        ),
        BIC_drive_value = case_when(
            (alignuniquedifferentiators_score >= 4 & presentimpactstory_score >= 4) |
            (alignuniquedifferentiators_score >= 4 & sharekeyinsights_score >= 4) |
            (presentimpactstory_score >= 4 & sharekeyinsights_score >= 4) ~ 1,
            TRUE ~ 0
        ),
        BIC_gain_commitment = case_when(
            (establishalignmentaccountability_score >= 4 & navigatedecisionmakingprocess_score >= 4) |
            (establishalignmentaccountability_score >= 4 & setpurposefulmilestones_score >= 4) |
            (navigatedecisionmakingprocess_score >= 4 & setpurposefulmilestones_score >= 4) ~ 1,
            TRUE ~ 0
        ),
        BIC_any = case_when(
            BIC_deliver_the_business_case == 1 |
            BIC_discovery == 1 |
            BIC_drive_value == 1 |
            BIC_gain_commitment == 1 ~ 1,
            TRUE ~ 0
        )
    ) %>%
    summarise(
        total_calls = n(),
        deliver_the_business_case = sum(BIC_deliver_the_business_case),
        discovery = sum(BIC_discovery),
        drive_value = sum(BIC_drive_value),
        gain_commitment = sum(BIC_gain_commitment),
        any_BIC = sum(BIC_any)
    ) %>%
    transmute(
        total_calls,
        any_BIC,
        any_BIC_pct = round(100*any_BIC/total_calls, 2), 
        
        deliver_the_business_case,
        deliver_the_business_case_pct = round(100*deliver_the_business_case/total_calls, 2),

        discovery,
        discovery_pct = round(100*discovery/total_calls, 2),

        drive_value,
        drive_value_pct = round(100*drive_value/total_calls, 2),

        gain_commitment,
        gain_commitment_pct = round(100*gain_commitment/total_calls, 2)
    )


Fraction of calls which are good/foundational/best-in-class

In [63]:
scores <- data %>% 
  select(-c(`...1`, crm_country, rep_country, rep_region, calldate, emailaddress, 
            multithreading_score, level_03_manager_name, level_04_manager_name,
           role, job_title, job_family, calldate, title, duration, fiscal_quarter,
           sales_region)) %>% 
  pivot_longer(cols = 2:13, names_to = "topic") %>% 
  group_by(callid) %>% 
  summarise(av_score = mean(value, na.rm = T)) %>% 
  ungroup() %>% 
  select(-callid)

zero_to_one_count <- nrow(scores[scores$av_score < 1,])
one_to_two_count <- nrow(scores[scores$av_score >= 1 & scores$av_score < 2,])
two_to_three_count <- nrow(scores[scores$av_score >= 2 & scores$av_score < 3,])
three_to_four_count <- nrow(scores[scores$av_score >= 3 & scores$av_score < 4,])
four_to_five_count <- nrow(scores[scores$av_score >= 4,])

scores_together <- data.frame(
  values = c(zero_to_one_count, one_to_two_count, two_to_three_count, three_to_four_count, four_to_five_count),
  labels = c("No Score", "Skill Not Demonstrated", "Foundational", "Good", "Best in Class")
) %>% 
  mutate(percentage = paste0(signif(100* values/sum(values),2)," %"))




scores_together$labels  <- factor(scores_together$labels, 
                                  levels = rev(c("No Score", "Skill Not Demonstrated", "Foundational", "Good", "Best in Class")))


scores_together %>%  
  ggplot(aes(x = "", y = values, fill = labels, label = percentage)) + 
  geom_col(width = .25, color = "black") + 
  scale_fill_manual(values = rev(c("No Score" = rgb(242/255,242/255,242/255), 
                               "Skill Not Demonstrated" = rgb(222/255,241/255,211/255), 
                               "Foundational" = rgb(159/255,215/255,127/255), 
                               "Good" = rgb(76/255,123/255,49/255), 
                               "Best in Class" = li_black))) +
  coord_flip() +
  theme_void() +
  labs(fill = "")

DT::datatable(scores_together)

Average subskill scores for each region

In [67]:
 data %>% 
  group_by(rep_region)

In [70]:
(av_data_region <- 
  data %>% 
  group_by(rep_region) %>% 
  dplyr::summarize(av_research = mean(research_score, na.rm = T),
            av_high_impact_questions = mean(highimpactquestions_score, na.rm = T),
            av_examine_challenges = mean(examinechallenges_score, na.rm = T),
            av_impact_story = mean(presentimpactstory_score, na.rm = T),
            av_key_insights = mean(sharekeyinsights_score, na.rm = T),
            av_unique_differentiators = mean(alignuniquedifferentiators_score, na.rm = T),
            av_multithreading = mean(multithreading_score, na.rm = T),
            av_compelling_proposals = mean(delivercompellingproposals_score, na.rm = T),
            av_deal_negotiate = mean(dealnegotiate_score, na.rm = T),
            av_handle_objections = mean(handleobjections_score, na.rm = T),
            av_navigate_dm_process = mean(navigatedecisionmakingprocess_score, na.rm = T),
            av_purposeful_milestones = mean(setpurposefulmilestones_score, na.rm = T),
            av_establish_alignment = mean(establishalignmentaccountability_score, na.rm = T)))

Heatmap of average skill scores

In [71]:
vals <- c(0,1,2,3,4,5)
newvals <- scales::rescale(vals, to = 0:1)

av_data_region %>% 
  pivot_longer(cols = 2:14, names_to = "topic") %>% 
  ggplot(aes(x = rep_region, y = topic, fill = value)) +
    geom_tile() +
    coord_equal() +
    labs(fill = "Av. Score\n", y = "", x = "") +
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),
          axis.text.y = element_text(vjust = 0.4),
          panel.background = element_rect(fill = "white"),
          text = element_text(size = 8),
          legend.title = element_text(size = 8),
          legend.text = element_text(size = 7)) +
    scale_fill_gradientn(colours=rev(cs), values = newvals, breaks=vals, limits = c(0,5)) +
    scale_y_discrete(labels= c(
      "av_unique_differentiators" = "Align Unique Differentiators",
      "av_research" = "Research",
      "av_purposeful_milestones" = "Set purposeful milestones",
      "av_deal_negotiate" = "Deal Negotiation",
      "av_navigate_dm_process" = "Navigate DM Process",
      "av_multithreading" = "Multithreading",
      "av_key_insights" = "Share Key Insights",
      "av_impact_story" = "Present the Impact Story",
      "av_high_impact_questions" = "High-Impact Questions",
      "av_handle_objections" = "Handle Objections",
      "av_examine_challenges" = "Examine Challenges",
      "av_establish_alignment" = "Establish Alignment and Mutual Accountability ",
      "av_compelling_proposals" = "Deliver Compelling Proposals"
    ))


Normalised proficiency scores can be ported to excel if you prefer to create charts there

In [73]:
norm_data <- data %>% 
    select(-c(`...1`, crm_country, rep_country, rep_region, calldate, callid,
              multithreading_score, level_03_manager_name, level_04_manager_name,
              role, job_title, job_family, calldate, title, duration,
             fiscal_quarter, sales_region)) %>%
    pivot_longer(cols = 2:13, names_to = "topic") %>% 
  filter(value != 0) %>% # not interested in calls where the score isn't relevant
  
  group_by(emailaddress, topic, value) %>%
  summarise(
    cnt = n()
  ) %>%
  mutate(freq = cnt / sum(cnt)) %>%
  ungroup() %>%
  group_by(topic, value) %>%
  summarise(
    av = mean(freq)
  ) %>%
   
  mutate(norm_score = av/sum(av)*100) %>% 
  select(topic, value, norm_score)

norm_data$value <- factor(norm_data$value, levels = c("1","2","3","4","5"))

(norm_data <- 
  norm_data %>% 
  mutate(
    graph = case_when(
      str_detect(topic, "establishalignmentaccountability_score|setpurposefulmilestones_score|navigatedecisionmakingprocess_score") ~ "Gain Commitment",
      str_detect(topic, "research_score|highimpactquestions_score|examinechallenges_score") ~ "Discovery",
      str_detect(topic, "presentimpactstory_score|sharekeyinsights_score|alignuniquedifferentiators_score") ~ "Drive Value",
      str_detect(topic, "delivercompellingproposals_score|dealnegotiate_score|handleobjections_score") ~ "Deliver the Business Case",
      TRUE ~ "Multithreading" # we do not need to make a chart of multithreading
    )
  ) %>% filter(graph != "no") %>% arrange(graph, topic, value))


# Proficiency Charts

In [74]:

norm_data %>%
  filter(graph == "Deliver the Business Case") %>% 
  ggplot(aes(x = value, y = norm_score, group = topic, colour = topic)) +
    geom_point() + 
    geom_xspline(spline_shape = -0.3, show.legend = FALSE) +
    theme_classic() +
    theme(
      strip.text = element_text(color = "black"),
      strip.background = element_rect(colour="black", fill="white"),
      panel.spacing = unit(0.5, "cm"),
      legend.title = element_blank(),
      axis.text = element_text(size = 13),
      axis.title = element_text(size = 14),
      legend.position = "bottom",
      plot.title = element_text(hjust = 0.5),
      panel.grid.major.x =  element_line(color = li_grey,
                                          size = 0.5),
    ) +
    labs(y = "Proficiency %", x = "\nScore", col = "Topic", colour = "Topic", linetype = "Topic", shape = "Topic", title = "Deliver the Business Case") +
    scale_x_discrete(expand = c(0,0), limits = c("1", "2", "3", "4", "5")) +
    scale_y_continuous(expand = c(0,0), limits = c(0,100)) +
    scale_color_manual(values = c(li_orange, li_blue, li_red), labels = c(
      "handle_objections" = "Handle Objections", 
      "dealnegotiate_score" = "Deal Negotiation",
      "deliver_compelling_proposals" = "Deliver Compelling Proposals")
    ) +
    scale_linetype_manual(values = c(
      "handle_objections" = 1,
      "prove_roi" = 2,
      "deliver_compelling_proposals" = 1
    )) +
    #guides(color = guide_legend(override.aes = list(shape = 15, size = 6))) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = "                   \n                   "), 
               inherit.aes = FALSE,
               size = 6,
               fill = c(rgb(242/255,242/255,242/255), 
                        rgb(222/255,241/255,211/255), 
                        rgb(159/255,215/255,127/255), 
                        rgb(76/255,123/255,49/255)),
               label.size = NA) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = label), 
             inherit.aes = FALSE, size = 5,
             fill = NA, label.size = NA,
             color = c(rep("black", 3), "white"))


In [75]:
norm_data %>%
  filter(graph == "Discovery") %>% 
  ggplot(aes(x = value, y = norm_score, group = topic, colour = topic)) +
    geom_point() + 
    geom_xspline(spline_shape = -0.3, show.legend = FALSE) +
    theme_classic() +
    theme(
      strip.text = element_text(color = "black"),
      strip.background = element_rect(colour="black", fill="white"),
      panel.spacing = unit(0.5, "cm"),
      legend.title = element_blank(),
      axis.text = element_text(size = 13),
      axis.title = element_text(size = 14),
      legend.position = "bottom",
      plot.title = element_text(hjust = 0.5),
            panel.grid.major.x =  element_line(color = li_grey,
                                          size = 0.5)
    ) +
    labs(y = "Proficiency %", x = "\nScore", col = "Topic", colour = "Topic", linetype = "Topic", shape = "Topic", title = "Discovery") +
    scale_x_discrete(expand = c(0,0), limits = c("1", "2", "3", "4", "5")) +
    scale_y_continuous(expand = c(0,0), limits = c(0,100)) +
    scale_color_manual(values = c(li_orange, li_blue, li_red), labels = c(
      "research_score" = "Research", 
      "examinechallenges_score" = "Examines Challenges",
      "highimpactquestions_score" = "High Impact Questions")
    ) +
    scale_linetype_manual(values = c(
      "research_score" = 2,
      "examinechallenges_score" = 1,
      "highimpactquestions_score" = 1
    )) +
   guides(color = guide_legend(override.aes = list(shape = 15, size = 6))) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = "                   \n                   "), 
               inherit.aes = FALSE,
               size = 6,
               fill = c(rgb(242/255,242/255,242/255), 
                        rgb(222/255,241/255,211/255), 
                        rgb(159/255,215/255,127/255), 
                        rgb(76/255,123/255,49/255)),
               label.size = NA) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = label), 
             inherit.aes = FALSE, size = 5,
             fill = NA, label.size = NA,
             color = c(rep("black", 3), "white"))

In [76]:
norm_data %>%
  filter(graph == "Drive Value") %>% 
  ggplot(aes(x = value, y = norm_score, group = topic, colour = topic)) +
    geom_xspline(spline_shape = -0.3, show.legend = FALSE) + 
    geom_point() +
    theme_classic() +
    theme(
      strip.text = element_text(color = "black"),
      strip.background = element_rect(colour="black", fill="white"),
      panel.spacing = unit(0.5, "cm"),
      legend.title = element_blank(),
      axis.text = element_text(size = 13),
      axis.title = element_text(size = 14),
      legend.position = "bottom",
      plot.title = element_text(hjust = 0.5),
            panel.grid.major.x =  element_line(color = li_grey,
                                          size = 0.5)
    ) +
    labs(y = "Proficiency %", x = "\nScore", col = "Topic", colour = "Topic", linetype = "Topic", shape = "Topic", title = "Drive Value") +
    scale_x_discrete(expand = c(0,0), limits = c("1", "2", "3", "4", "5")) +
    scale_y_continuous(expand = c(0,0), limits = c(0,100)) +
    scale_color_manual(values = c(li_orange, li_blue, li_red), labels = c(
      "align_unique_differentiators" = "Align Unique Differentiators", 
      "present_impact_story" = "Impact Story",
      "share_key_insights" = "Share Key Insights")
    ) +
    scale_linetype_manual(values = c(
      "align_unique_differentiators" = 1,
      "present_impact_story" = 1,
      "share_key_insights" = 1
    )) +
   guides(color = guide_legend(override.aes = list(shape = 15, size = 6))) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = "                   \n                   "), 
               inherit.aes = FALSE,
               size = 6,
               fill = c(rgb(242/255,242/255,242/255), 
                        rgb(222/255,241/255,211/255), 
                        rgb(159/255,215/255,127/255), 
                        rgb(76/255,123/255,49/255)),
               label.size = NA) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = label), 
             inherit.aes = FALSE, size = 5,
             fill = NA, label.size = NA,
             color = c(rep("black", 3), "white"))

In [77]:
norm_data %>%
  filter(graph == "Gain Commitment") %>% 
  ggplot(aes(x = value, y = norm_score, group = topic, colour = topic)) +
    geom_point() + 
    geom_xspline(spline_shape = -0.3, show.legend = FALSE) +
    theme_classic() +
    theme(
      strip.text = element_text(color = "black"),
      strip.background = element_rect(colour="black", fill="white"),
      panel.spacing = unit(0.5, "cm"),
      legend.title = element_blank(),
      axis.text = element_text(size = 13),
      axis.title = element_text(size = 14),
      legend.position = "bottom",
      plot.title = element_text(hjust = 0.5),
            panel.grid.major.x =  element_line(color = li_grey,
                                          size = 0.5)
    ) +
    labs(y = "Proficiency %", x = "\nScore", col = "Topic", colour = "Topic", linetype = "Topic", shape = "Topic", title = "Gain Commitment") +
    scale_x_discrete(expand = c(0,0), limits = c("1", "2", "3", "4", "5")) +
    scale_y_continuous(expand = c(0,0), limits = c(0,100)) +
    scale_color_manual(values = c(li_orange, li_blue, li_red), labels = c(
      "establish_alignment_accountability" = "Establish Accountability", 
      "navigate_decisionmaking_processes" = "Navigating Decision Making Process",
      "set_purposeful_milestones" = "Set Purposeful Milestones")
    ) +
    scale_linetype_manual(values = c(
      "establish_alignment_accountability" = 1,
      "navigate_decisionmaking_processes" = 1,
      "set_purposeful_milestones" = 1
    )) +
   guides(color = guide_legend(override.aes = list(shape = 15, size = 6))) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = "                   \n                   "), 
               inherit.aes = FALSE,
               size = 6,
               fill = c(rgb(242/255,242/255,242/255), 
                        rgb(222/255,241/255,211/255), 
                        rgb(159/255,215/255,127/255), 
                        rgb(76/255,123/255,49/255)),
               label.size = NA) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = label), 
             inherit.aes = FALSE, size = 5,
             fill = NA, label.size = NA,
             color = c(rep("black", 3), "white"))

In [34]:
norm_data %>%
  filter(graph == "Multithreading") %>% 
  ggplot(aes(x = value, y = norm_score, group = topic)) +
    geom_point() + 
    geom_xspline() + 
    theme_classic() +
    theme(
      strip.text = element_text(color = "black"),
      strip.background = element_rect(colour="black", fill="white"),
      panel.spacing = unit(0.5, "cm"),
      legend.title = element_blank(),
      axis.text = element_text(size = 13),
      axis.title = element_text(size = 14),
      legend.position = "bottom",
      plot.title = element_text(hjust = 0.5),
            panel.grid.major.x =  element_line(color = li_grey,
                                          size = 0.5)
    ) +
    labs(y = "Proficiency %", x = "\nScore", col = "Topic", colour = "Topic", linetype = "Topic", shape = "Topic", title = "Multithreading") +
    scale_x_discrete(expand = c(0,0), limits = c("1", "2", "3", "4", "5")) +
    scale_y_continuous(expand = c(0,0), limits = c(0,100)) +
    scale_color_manual(values = c(li_orange), labels = c(
      "multithreading_score" = "Multithreading"
      )
    ) +
    scale_linetype_manual(values = c(
      "multithreading_score" = 1
    )) +
   guides(color = guide_legend(override.aes = list(shape = 15, size = 6))) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = "                   \n                   "), 
               inherit.aes = FALSE,
               size = 6,
               fill = c(rgb(242/255,242/255,242/255), 
                        rgb(222/255,241/255,211/255), 
                        rgb(159/255,215/255,127/255), 
                        rgb(76/255,123/255,49/255)),
               label.size = NA) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = label), 
             inherit.aes = FALSE, size = 5,
             fill = NA, label.size = NA,
             color = c(rep("black", 3), "white"))

In [78]:
norm_data %>% 
  group_by(graph, value) %>% 
  filter(graph != "Multithreading") %>% 
  summarise(averaged_score = mean(norm_score)) %>% 
  mutate(norm_av = averaged_score/sum(averaged_score)*100) %>% 
  select(-averaged_score) %>% 
  ggplot(aes(x = value, y = norm_av, color = graph, group = graph)) +
    geom_point() + 
    geom_xspline(spline_shape = -0.3, show.legend = FALSE) +
    theme_classic() +
    labs(title = "Skill-Based Proficiency Curve") +
    theme(legend.position = "bottom", 
          legend.title = element_blank(),
          plot.title = element_text(hjust = 0.5),
                panel.grid.major.x =  element_line(color = li_grey,
                                          size = 0.5),
      axis.text = element_text(size = 13),
      axis.title = element_text(size = 14)
          ) +
    labs(y = "Proficiency %", x = "\nScore", col = "Topic") +
    scale_x_discrete(expand = c(0,0), limits = c("1", "2", "3", "4", "5")) +
    scale_y_continuous(expand = c(0,0), limits = c(0,100)) +
    guides(color = guide_legend(override.aes = list(shape = 15, size = 6))) +
    scale_color_manual(values = c(li_orange, li_blue, li_red, li_green)) +
  
     geom_label(data = annotation, aes(x = x, y = y, label = "                   \n                   "), 
               inherit.aes = FALSE,
               size = 6,
               fill = c(rgb(242/255,242/255,242/255), 
                        rgb(222/255,241/255,211/255), 
                        rgb(159/255,215/255,127/255), 
                        rgb(76/255,123/255,49/255)),
               label.size = NA) +
  
    geom_label(data = annotation, aes(x = x, y = y, label = label), 
             inherit.aes = FALSE, size = 5,
             fill = NA, label.size = NA,
             color = c(rep("black", 3), "white"))


In [79]:
norm_data <- data %>% 
    select(-c(`...1`, crm_country, rep_country, callid, calldate, 
              level_03_manager_name, level_04_manager_name,
              role, job_title, job_family, calldate, title, duration,
             fiscal_quarter, sales_region)) %>%
  pivot_longer(cols = 3:15, names_to = "topic") %>% 
  filter(value != 0) %>% 
  
  group_by(rep_region, emailaddress, topic, value) %>%
   
  summarise(
    cnt = n()
  ) %>%
   
  mutate(freq = cnt / sum(cnt)) %>%
   
  ungroup() %>%
  
  group_by(rep_region, topic, value) %>%
  summarise(
    av = mean(freq)
  ) %>%
   
  mutate(norm_score = av/sum(av)*100) %>% 
  select(topic, value, norm_score, rep_region)

norm_data$value <- factor(norm_data$value, levels = c("1","2","3","4","5"))

norm_data <- 
  norm_data %>% 
  mutate(
    graph = case_when(
      str_detect(topic, "establishalignmentaccountability_score|setpurposefulmilestones_score|navigatedecisionmakingprocess_score") ~ "Gain Commitment",
      str_detect(topic, "research_score|highimpactquestions_score|examinechallenges_score") ~ "Discovery",
      str_detect(topic, "presentimpactstory_score|sharekeyinsights_score|alignuniquedifferentiators_score") ~ "Drive Value",
      str_detect(topic, "delivercompellingproposals_score|dealnegotiate_score|handleobjections_score") ~ "Deliver the Business Case",
      TRUE ~ "Multithreading" # we do not need to make a chart of multithreading
    )
  ) %>% filter(graph != "no")

norm_data %>%
  ggplot(aes(x = value, y = norm_score, color = topic, group = topic)) +
  facet_grid(rows = vars(graph), cols = vars(rep_region)) +
  geom_xspline(spline_shape = -0.3, show.legend = FALSE) +
  theme_linedraw() +
  theme(
    strip.text = element_text(color = "black"),
    strip.background = element_rect(colour="black", fill="white"),
    panel.spacing = unit(0.5, "cm")
  ) +
  labs(y = "Proficiency %", x = "Score", color = "") +
  scale_x_discrete(expand = c(0,0)) +
  scale_y_continuous(expand = c(0,0), limits = c(0,100)) 

Call classifaction aggregated by manager

In [82]:
scores <- data %>% 
  select(-c(`...1`, crm_country, rep_country, rep_region, calldate, 
            emailaddress, multithreading_score, level_03_manager_name, 
            role, job_title, job_family, title, duration, fiscal_quarter,
           sales_region)) %>% 
  pivot_longer(cols = 3:14, names_to = "topic") %>% 
  group_by(callid, level_03_manager_name) %>% 
  summarise(av_score = mean(value)) %>% 
  ungroup() %>% 
  select(-callid) %>% 
  mutate(
    tier = case_when(
      av_score < 1 ~ "No Score",
      av_score >= 1 & av_score < 2 ~ "Skill Not Demonstrated",
      av_score >= 2 & av_score < 3 ~ "Foundational",
      av_score >= 3 & av_score < 4 ~ "Good",
      av_score >= 4 ~ "Best in Class",
      TRUE ~ "ERROR"
    )
  ) %>% 
  select(-av_score)

scores$tier <- factor(scores$tier, levels = rev(c("No Score", "Skill Not Demonstrated", 
                                              "Foundational", "Good", "Best in Class",
                                              "ERROR")))


ggplot(data = scores, aes(y = tier, fill = level_03_manager_name)) +
  geom_bar(position = "stack") +
  theme_linedraw() +
  labs(y = "")

DT::datatable(scores %>% group_by(level_03_manager_name, tier) %>% summarise(count = n()))